In [32]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from math import factorial
import numpy as np
from scipy import stats as st

In [33]:
visitas = pd.read_csv('datos/visits_log_us.csv')

In [34]:
pedidos = pd.read_csv('datos/orders_log_us.csv')

In [35]:
gastos = pd.read_csv('datos/costs_us.csv')

VISITAS 

In [43]:
visitas.info()

<class 'pandas.core.series.Series'>
Index: 1 entries, Device to Device
Series name: None
Non-Null Count  Dtype  
--------------  -----  
1 non-null      float64
dtypes: float64(1)
memory usage: 124.0+ bytes


In [44]:
visitas.head()

Device    0.0
dtype: float64

In [38]:
visitas = visitas.isnull().mean() * 100
print(visitas)

Device       0.0
End Ts       0.0
Source Id    0.0
Start Ts     0.0
Uid          0.0
dtype: float64


In [39]:
visitas = visitas.drop_duplicates()
print(visitas.duplicated().sum())
visitas.reset_index(drop=True)

0


0    0.0
dtype: float64

Convertiremos las columnas 'Start Ts' y 'End Ts' a formato datetime para poder trabajar con ellas

In [47]:
visitas['Start Ts'] = pd.to_datetime(visitas['Start Ts'])
visitas['End Ts'] = pd.to_datetime(visitas['End Ts'])

KeyError: 'Start Ts'

Ahora Trabajaremos para encontrar la cantidad de usuarios por día, semana, mes

In [ ]:
u_dia = visitas.groupby(visitas['Start Ts'].dt.date)['Uid'].nunique()
u_semana = visitas.groupby(visitas['Start Ts'].dt.to_period('W'))['Uid'].nunique()
u_mes = visitas.groupby(visitas['Start Ts'].dt.to_period('M'))['Uid'].nunique()

print(u_dia.head(5))
print(u_semana.head(5))
print(u_mes.head(5))

Start Ts
2017-06-01    605
2017-06-02    608
2017-06-03    445
2017-06-04    476
2017-06-05    820
Name: Uid, dtype: int64
Start Ts
2017-05-29/2017-06-04    2021
2017-06-05/2017-06-11    4129
2017-06-12/2017-06-18    2812
2017-06-19/2017-06-25    2878
2017-06-26/2017-07-02    3064
Freq: W-SUN, Name: Uid, dtype: int64
Start Ts
2017-06    13259
2017-07    14183
2017-08    11631
2017-09    18975
2017-10    29692
Freq: M, Name: Uid, dtype: int64


Revisaremos cuantas sesiones se realizan por día.

In [ ]:
ses_por_dia = visitas.groupby(visitas['Start Ts'].dt.date)['Uid'].count()
print("Sesiones por dia:\n", ses_por_dia.head(5))

Sesiones por dia:
 Start Ts
2017-06-01    664
2017-06-02    658
2017-06-03    477
2017-06-04    510
2017-06-05    893
Name: Uid, dtype: int64


Revisaremos si un usuario ingresa más de una vez

In [ ]:
ses_por_us = visitas.groupby('Uid')['Start Ts'].count()
multiple_ses_us = ses_por_us[ses_por_us > 1]

print("Usuarios con más de una Sesión por Día:\n", multiple_ses_us)

Usuarios con más de una Sesión por Día:
 Uid
313578113262317         3
325320750514679         2
526778907996220         4
577434573913691         2
1260655184775459        4
                       ..
18445884613277162497    3
18446104389491815722    2
18446156210226471712    7
18446167067214817906    2
18446621818809592527    2
Name: Start Ts, Length: 52128, dtype: int64


Vamos a revisar cuanto dura cada sesión del visitante

In [ ]:
visitas['duracion'] = visitas['End Ts'] - visitas['Start Ts']
print("Duracion de Cada Sesion:\n", visitas['duracion'])

Duracion de Cada Sesion:
 0        0 days 00:18:00
1        0 days 00:28:00
2        0 days 00:00:00
3        0 days 00:24:00
4        0 days 00:00:00
               ...      
359395   0 days 00:00:19
359396   0 days 00:00:19
359397   0 days 00:00:19
359398   0 days 00:00:19
359399   0 days 00:00:19
Name: duracion, Length: 359400, dtype: timedelta64[ns]


Revisaremos con que frecuencia regresa al sitio cada usuario

In [ ]:
visitas['Start Ts'] = pd.to_datetime(visitas['Start Ts'])
visitas = visitas.sort_values(by=['Uid','Start Ts'])
frecuencia_regreso = visitas.groupby('Uid')['Start Ts'].diff()
print(frecuencia_regreso)

196543                 NaT
257333                 NaT
173818                 NaT
173142                 NaT
192554   152 days 16:28:00
                ...       
175259                 NaT
293449                 NaT
319162                 NaT
222173   144 days 20:03:00
132805                 NaT
Name: Start Ts, Length: 359400, dtype: timedelta64[ns]


Crearemos un gráfico

PEDIDOS

In [ ]:
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


In [ ]:
pedidos.head(5)

,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450


In [ ]:
print(pedidos.isnull().sum())

Buy Ts     0
Revenue    0
Uid        0
dtype: int64


In [ ]:
pedidos = pedidos.drop_duplicates()
print(pedidos.duplicated().sum())
pedidos.reset_index(drop=True)

0


,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450
...,...,...,...
50410,2018-05-31 23:50:00,4.64,12296626599487328624
50411,2018-05-31 23:50:00,5.80,11369640365507475976
50412,2018-05-31 23:54:00,0.30,1786462140797698849
50413,2018-05-31 23:56:00,3.67,3993697860786194247


Haremos un Histograma por mes

Convertiremos la columna 'Buy Ts' a formato datetime para poder trabajar con ella

In [41]:
pedidos['Buy Ts'] = pd.to_datetime(pedidos['Buy Ts'])

Vamos a obtener la primera Orden

In [46]:
primera_orden = pedidos.groupby('Uid')['Buy Ts'].min()
print(primera_orden.head())

Uid
313578113262317    2018-01-03 21:51:00
1575281904278712   2017-06-03 10:13:00
2429014661409475   2017-10-11 18:33:00
2464366381792757   2018-01-28 15:54:00
2551852515556206   2017-11-24 10:14:00
Name: Buy Ts, dtype: datetime64[ns]


Vamos a obtener

In [48]:
primera_orden = primera_orden.to_frame(name='primera_compra')
pedidos = pedidos.merge(primera_orden, left_on='Uid', right_index=True, how='left')
print(pedidos.head())

               Buy Ts  Revenue                   Uid      primera_compra
0 2017-06-01 00:10:00    17.00  10329302124590727494 2017-06-01 00:10:00
1 2017-06-01 00:25:00     0.55  11627257723692907447 2017-06-01 00:25:00
2 2017-06-01 00:27:00     0.37  17903680561304213844 2017-06-01 00:27:00
3 2017-06-01 00:29:00     0.55  16109239769442553005 2017-06-01 00:29:00
4 2017-06-01 07:58:00     0.37  14200605875248379450 2017-06-01 07:58:00


Vamos a trabajar para obtener los cohortes Mensual

In [49]:
pedidos['primera_orden_mes'] = pedidos['primera_compra'].astype('datetime64[M]')
pedidos['orden_mes'] = pedidos['Buy Ts'].astype('datetime64[M]')

print(pedidos.head())

TypeError: Cannot cast DatetimeArray to dtype datetime64[M]

In [50]:
grupo_cohort = pedidos.groupby('primera_orden_mes').agg({'Uid': 'nunique', 'Revenue': 'sum'})
print(grupo_cohort.head())

KeyError: 'primera_orden_mes'

Podemos observar que los ingresos por cohortes bajaron, se recuperaron pero nuevamente bajaron y otra vez hay un repunte

Realizaremos una tabla Dinámica para conocer que clientes siguen realizando compras despues de que realizaron la primera

In [51]:
pedidos.pivot_table(
    index='primera_orden_mes',
    columns='orden_mes',
    values='Uid',
    aggfunc='nunique',)

KeyError: 'primera_orden_mes'

Existe una disminución de clientes despues del primer mes, luego se recupera y nuevamente hay una caida y otra vez se recupera.

Haremos un gráfico de lineas

Vamos a calcular el promedio de compra(ingresos)

In [52]:
pedidos_agrupados_cohorts = pedidos.groupby( ['primera_orden_mes', 'orden_mes']).agg({'Revenue': 'sum', 'Uid': 'nunique'})
print(pedidos_agrupados_cohorts.head())

KeyError: 'primera_orden_mes'

In [ ]:
pedidos_agrupados_cohorts.columns

Buscaremos la compra por usuario

In [ ]:
pedidos_agrupados_cohorts['Ingreso_por_uid'] = (pedidos_agrupados_cohorts['Revenue']
    / pedidos_agrupados_cohorts['Uid'])

Realizaremos una Tabla Dinámica para observar la compra mensual por usuario.

In [ ]:
pedidos_agrupados_cohorts.pivot_table(
    index='primera_orden_mes',
    columns='orden_mes',
    values='Ingreso_por_uid',
    aggfunc='mean',)